In [69]:
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

# Set up the WebDriver with the start-maximized option
options = webdriver.EdgeOptions()
options.add_argument("start-maximized")
driver = webdriver.Edge(options=options)

location = input("Enter the location of the file: ")
jobtype = input("Enter the job type: ")

url = f"https://www.linkedin.com/jobs/{jobtype}-jobs/?location={location}&trk=guest_job_search_jobs-search-bar_search-submit&redirect=false&position=1&pageNum=0&currentJobId=3795333449"
https://www.linkedin.com/jobs/{job_title}-jobs/?location={location}

driver.get(url)

total_jobs = 20
job_titles = []
job_companies = []
job_locations = []
job_links = []
job_descriptions = []

print("Scraping jobs...")

# Infinite scroll
while len(job_titles) < total_jobs:
    
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait for page to load
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # Parse page source with BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Extract data
    job_title_elements = driver.find_elements(by="class name", value="base-search-card__title")
    job_titles += [element.text for element in job_title_elements]

    job_company_elements = driver.find_elements(by="class name", value="base-search-card__subtitle")
    job_companies += [element.text for element in job_company_elements]

    job_location_elements = driver.find_elements(by="class name", value="job-search-card__location")
    job_locations += [element.text for element in job_location_elements]

    job_link_elements = driver.find_elements(by="class name", value="base-card__full-link")
    job_links += [element.get_attribute("href") for element in job_link_elements]

    print(f'Total Jobs Extracted: {len(job_titles)}')
    
    # If we've collected enough jobs, break the loop
    if len(job_titles) >= total_jobs:
        print("Reached the desired number of jobs. Stopping...")
        break

    try:
        # Use WebDriverWait with ExpectedConditions to wait for the button to be clickable
        button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "infinite-scroller__show-more-button"))
        )
        button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "infinite-scroller__show-more-button"))
        )
        button.click()
        print("Clicking 'Show More' button...")
    except (NoSuchElementException, ElementNotInteractableException):
        # No more "Show More" button or button not interactable, break the loop
        print("No more 'Show More' button. Stopping...")
        break


# Extract job descriptions
for job_link in job_links:
    driver.get(job_link)
    
    # Handle authentication wall
    
    for _ in range(3):  # Try a few times
        try:
            show_more_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "show-more-less-html__button"))
            )
            show_more_button.click()
            print("Clicking 'Show more' button for job description...")
            
            # Wait for the job description to load
            time.sleep(2)
            
            description_element = driver.find_element(By.CLASS_NAME, "show-more-less-html__markup")
            job_descriptions.append(description_element.text)
            print("Job description extracted.")
            break  # Exit the loop if successful
        except TimeoutException:
            job_descriptions.append("N/A")
            print("TimeoutException: No 'Show more' button found for job description within the specified timeout.")
            # If authentication wall detected, refresh the page
            if "authwall" in driver.current_url:
                print("Authentication wall detected. Pausing for 5 seconds...")
                time.sleep(5)
                print("Authentication wall detected. Searching the job again...")
                driver.get(job_link)
                
        except (NoSuchElementException, ElementNotInteractableException):
            job_descriptions.append("N/A")
            print("No 'Show more' button found for job description.")
            break  # Exit the loop if the button is not found.

# Trim lists to desired length
job_titles = job_titles[:total_jobs]
job_companies = job_companies[:total_jobs]
job_locations = job_locations[:total_jobs]
job_links = job_links[:total_jobs]
job_descriptions = job_descriptions[:total_jobs]

# Store in DataFrame
df = pd.DataFrame({
    'Job Title': job_titles,
    'Company': job_companies,
    'Location': job_locations,
    'Link': job_links,
    'Description': job_descriptions
})

# Save to CSV
df.to_csv('jobs_with_description.csv', index=False)

# Close the WebDriver
driver.quit()

print("Job scraping completed. Check 'jobs_with_description.csv' for the results.")


The msedgedriver version (120.0.2210.133) detected in PATH at d:\Project\Python_Scraping\msedgedriver.exe might not be compatible with the detected MicrosoftEdge version (121.0.2277.49); currently, msedgedriver 121.0.2277.49 is recommended for MicrosoftEdge 121.*, so it is advised to delete the driver in PATH and retry


Scraping jobs...
Total Jobs Extracted: 50
Reached the desired number of jobs. Stopping...
Clicking 'Show more' button for job description...
Job description extracted.
Clicking 'Show more' button for job description...
Job description extracted.
Clicking 'Show more' button for job description...
Job description extracted.
TimeoutException: No 'Show more' button found for job description within the specified timeout.
Authentication wall detected. Pausing for 5 seconds...
Authentication wall detected. Searching the job again...
Clicking 'Show more' button for job description...
Job description extracted.
Clicking 'Show more' button for job description...
Job description extracted.
Clicking 'Show more' button for job description...
Job description extracted.
TimeoutException: No 'Show more' button found for job description within the specified timeout.
Authentication wall detected. Pausing for 5 seconds...
Authentication wall detected. Searching the job again...
Clicking 'Show more' butto

Traceback (most recent call last):
  File "C:\Users\akver\AppData\Local\Temp\ipykernel_19704\3630421912.py", line 90, in <module>
    show_more_button = WebDriverWait(driver, 10).until(
  File "c:\Users\akver\anaconda3\envs\ai\lib\site-packages\selenium\webdriver\support\wait.py", line 105, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6869D0332+58194]
	Microsoft::Applications::Events::ILogConfiguration::operator* [0x00007FF68695EC92+216482]
	(No symbol) [0x00007FF686779E1B]
	(No symbol) [0x00007FF6867C9C6F]
	(No symbol) [0x00007FF6867C9DA5]
	(No symbol) [0x00007FF68680BB27]
	(No symbol) [0x00007FF6867EC11F]
	(No symbol) [0x00007FF6867BD3E1]
	(No symbol) [0x00007FF686808A7E]
	(No symbol) [0x00007FF6867EBD43]
	(No symbol) [0x00007FF6867BC162]
	(No symbol) [0x00007FF6867BB5BD]
	(No symbol) [0x00007FF6867BC994]
	Microsoft::Applications::Events::EventProperty::to_string [0x000

In [70]:
url 

'https://www.linkedin.com/jobs/Python-jobs/?location=India&trk=guest_job_search_jobs-search-bar_search-submit&redirect=false&position=1&pageNum=0'

In [ ]:
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

# Set up the WebDriver with the start-maximized option
options = webdriver.EdgeOptions()
options.add_argument("start-maximized")
driver = webdriver.Edge(options=options)

location = input("Enter the location of the file: ")
jobtype = input("Enter the job type: ")

url = f"https://www.linkedin.com/jobs/{jobtype}-jobs/?location={location}&trk=guest_job_search_jobs-search-bar_search-submit&redirect=false&position=1&pageNum=0&currentJobId=3795333449"

driver.get(url)

total_jobs = 20
job_titles = []
job_companies = []
job_locations = []
job_links = []
job_descriptions = []

print("Scraping jobs...")

# Infinite scroll
while len(job_titles) < total_jobs:
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait for page to load
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        if new_height == last_height:
            break
        
        last_height = new_height

    # Parse page source with BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Extract data
    job_title_elements = driver.find_elements(by="class name", value="base-search-card__title")
    job_titles += [element.text for element in job_title_elements]

    job_company_elements = driver.find_elements(by="class name", value="base-search-card__subtitle")
    job_companies += [element.text for element in job_company_elements]

    job_location_elements = driver.find_elements(by="class name", value="job-search-card__location")
    job_locations += [element.text for element in job_location_elements]

    job_link_elements = driver.find_elements(by="class name", value="base-card__full-link")
    job_links += [element.get_attribute("href") for element in job_link_elements]

    print(f'Total Jobs Extracted: {len(job_titles)}')
    
    # If we've collected enough jobs, break the loop
    if len(job_titles) >= total_jobs:
        print("Reached the desired number of jobs. Stopping...")
        break

    try:
        # Use WebDriverWait with ExpectedConditions to wait for the button to be clickable
        button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "infinite-scroller__show-more-button"))
        )
        button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "infinite-scroller__show-more-button"))
        )
        button.click()
        print("Clicking 'Show More' button...")
    except (NoSuchElementException, ElementNotInteractableException):
        # No more "Show More" button or button not interactable, break the loop
        print("No more 'Show More' button. Stopping...")
        break


# Extract job descriptions
for job_link in job_links:
    driver.get(job_link)
    
    # Handle authentication wall
    for _ in range(3):  # Try a few times
        try:
            show_more_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "show-more-less-html__button"))
            )
            show_more_button.click()
            print("Clicking 'Show more' button for job description...")
            
            # Wait for the job description to load
            time.sleep(2)
            
            description_element = driver.find_element(By.CLASS_NAME, "show-more-less-html__markup")
            job_descriptions.append(description_element.text)
            print("Job description extracted.")
            break  # Exit the loop if successful
        except TimeoutException:
            job_descriptions.append("N/A")
            print("TimeoutException: No 'Show more' button found for job description within the specified timeout.")
            # If authentication wall detected, refresh the page
            if "authwall" in driver.current_url:
                print("Authentication wall detected. Pausing for 5 seconds...")
                time.sleep(5)
                print("Authentication wall detected. Searching the job again...")
                driver.get(job_link)
        except (NoSuchElementException, ElementNotInteractableException):
            job_descriptions.append("N/A")
            print("No 'Show more' button found for job description.")
            break  # Exit the loop if the button is not found.

# Trim lists to the desired length
job_titles = job_titles[:total_jobs]
job_companies = job_companies[:total_jobs]
job_locations = job_locations[:total_jobs]
job_links = job_links[:total_jobs]
job_descriptions = job_descriptions[:total_jobs]

# Store in DataFrame
df = pd.DataFrame({
    'Job Title': job_titles,
    'Company': job_companies,
    'Location': job_locations,
    'Link': job_links,
    'Description': job_descriptions
})

# Save to CSV
df.to_csv('jobs_with_description.csv', index=False)

# Close the WebDriver
driver.quit()

print("Job scraping completed. Check 'jobs_with_description.csv' for the results.")


In [73]:
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, TimeoutException, StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

# Set up the WebDriver with the start-maximized option
options = webdriver.EdgeOptions()
options.add_argument("start-maximized")
driver = webdriver.Edge(options=options)

location = input("Enter the location of the file: ")
jobtype = input("Enter the job type: ")

url = f"https://www.linkedin.com/jobs/{jobtype}-jobs/?location={location}&trk=guest_job_search_jobs-search-bar_search-submit&redirect=false&position=1&pageNum=0&currentJobId=3795333449"

driver.get(url)

total_jobs = 20
job_details = []

print("Scraping jobs...")

# Infinite scroll
while len(job_details) < total_jobs:
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait for page to load
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        if new_height == last_height:
            break
        
        last_height = new_height

    # Parse page source with BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Extract data
    job_title_elements = driver.find_elements(by="class name", value="base-search-card__title")
    job_company_elements = driver.find_elements(by="class name", value="base-search-card__subtitle")
    job_location_elements = driver.find_elements(by="class name", value="job-search-card__location")
    job_link_elements = driver.find_elements(by="class name", value="base-card__full-link")

    for i in range(len(job_title_elements)):
        job_id = len(job_details) + 1

        # Handle StaleElementReferenceException
        try:
            job_title = job_title_elements[i].text if i < len(job_title_elements) else "N/A"
            job_company = job_company_elements[i].text if i < len(job_company_elements) else "N/A"
            job_location = job_location_elements[i].text if i < len(job_location_elements) else "N/A"
            job_link = job_link_elements[i].get_attribute("href") if i < len(job_link_elements) else "N/A"

            # Visit the job link to extract job description
            driver.get(job_link)
            
            try:
                # Handle authentication wall
                for _ in range(3):  # Try a few times
                    show_more_button = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.CLASS_NAME, "show-more-less-html__button"))
                    )
                    show_more_button.click()
                    print(f"Job {job_id}: Clicking 'Show more' button for job description...")

                    # Wait for the job description to load
                    time.sleep(2)

                    description_element = driver.find_element(By.CLASS_NAME, "show-more-less-html__markup")
                    job_description = description_element.text
                    print(f"Job {job_id}: Job description extracted.")
                    break  # Exit the loop if successful
            except TimeoutException:
                job_description = "N/A"
                print(f"Job {job_id}: TimeoutException - No 'Show more' button found for job description within the specified timeout.")
                # If authentication wall detected, refresh the page
                if "authwall" in driver.current_url:
                    print(f"Job {job_id}: Authentication wall detected. Pausing for 5 seconds...")
                    time.sleep(5)
                    print(f"Job {job_id}: Authentication wall detected. Searching the job again...")
                    driver.get(job_link)
            except (NoSuchElementException, ElementNotInteractableException):
                job_description = "N/A"
                print(f"Job {job_id}: No 'Show more' button found for job description.")
        except StaleElementReferenceException:
            print(f"Job {job_id}: StaleElementReferenceException - Retrying the element extraction...")
            continue

        job_details.append({
            'Job ID': job_id,
            'Job Title': job_title,
            'Company': job_company,
            'Location': job_location,
            'Link': job_link,
            'Description': job_description
        })

        print(f"Job {job_id}: Details extracted.")

    print(f'Total Jobs Extracted: {len(job_details)}')

    # If we've collected enough jobs, break the loop
    if len(job_details) >= total_jobs:
        print("Reached the desired number of jobs. Stopping...")
        break

    try:
        # Use WebDriverWait with ExpectedConditions to wait for the button to be clickable
        button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "infinite-scroller__show-more-button"))
        )
        button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "infinite-scroller__show-more-button"))
        )
        button.click()
        print("Clicking 'Show More' button...")
    except (NoSuchElementException, ElementNotInteractableException):
        # No more "Show More" button or button not interactable, break the loop
        print("No more 'Show More' button. Stopping...")
        break

# Store in DataFrame
df = pd.DataFrame(job_details)

# Save to CSV
df.to_csv('jobs_with_description.csv', index=False)

# Close the WebDriver
driver.quit()

print("Job scraping completed. Check 'jobs_with_description.csv' for the results.")


The msedgedriver version (120.0.2210.133) detected in PATH at d:\Project\Python_Scraping\msedgedriver.exe might not be compatible with the detected MicrosoftEdge version (121.0.2277.49); currently, msedgedriver 121.0.2277.49 is recommended for MicrosoftEdge 121.*, so it is advised to delete the driver in PATH and retry


Scraping jobs...
Job 1: Clicking 'Show more' button for job description...
Job 1: Job description extracted.
Job 1: Details extracted.
Job 2: StaleElementReferenceException - Retrying the element extraction...
Job 2: StaleElementReferenceException - Retrying the element extraction...
Job 2: StaleElementReferenceException - Retrying the element extraction...
Job 2: StaleElementReferenceException - Retrying the element extraction...
Job 2: StaleElementReferenceException - Retrying the element extraction...
Job 2: StaleElementReferenceException - Retrying the element extraction...
Job 2: StaleElementReferenceException - Retrying the element extraction...
Job 2: StaleElementReferenceException - Retrying the element extraction...
Job 2: StaleElementReferenceException - Retrying the element extraction...
Job 2: StaleElementReferenceException - Retrying the element extraction...
Job 2: StaleElementReferenceException - Retrying the element extraction...
Job 2: StaleElementReferenceException - 

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6869D0332+58194]
	Microsoft::Applications::Events::ILogConfiguration::operator* [0x00007FF68695EC92+216482]
	(No symbol) [0x00007FF686779E1B]
	(No symbol) [0x00007FF6867C9C6F]
	(No symbol) [0x00007FF6867C9DA5]
	(No symbol) [0x00007FF68680BB27]
	(No symbol) [0x00007FF6867EC11F]
	(No symbol) [0x00007FF6867BD3E1]
	(No symbol) [0x00007FF686808A7E]
	(No symbol) [0x00007FF6867EBD43]
	(No symbol) [0x00007FF6867BC162]
	(No symbol) [0x00007FF6867BB5BD]
	(No symbol) [0x00007FF6867BC994]
	Microsoft::Applications::Events::EventProperty::to_string [0x00007FF686B86CDA+1051354]
	(No symbol) [0x00007FF68683F596]
	Microsoft::Applications::Events::EventProperty::clear [0x00007FF6868D6215+34245]
	Microsoft::Applications::Events::EventProperty::clear [0x00007FF6868CE6BD+2669]
	Microsoft::Applications::Events::EventProperty::to_string [0x00007FF686B859D3+1046483]
	Microsoft::Applications::Events::ILogConfiguration::operator* [0x00007FF686

In [2]:
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

# Ask for user input
location = input("Enter the location of the file: ")
jobtype = input("Enter the job type: ")
total_jobs = int(input("Enter the number of jobs to scrape: "))
url = f"https://www.linkedin.com/jobs/{jobtype}-jobs/?location={location}"


# Set up the WebDriver with the start-maximized option
options = webdriver.EdgeOptions()
options.add_argument("start-maximized")
driver = webdriver.Edge(options=options)



driver.get(url)


job_titles = []
job_companies = []
job_locations = []
job_links = []
# job_descriptions = []

print("Scraping jobs...")

# Infinite scroll
while len(job_titles) < total_jobs:
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait for page to load
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        if new_height == last_height:
            break
        
        last_height = new_height

    # Parse page source with BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Extract data
    job_title_elements = driver.find_elements(by="class name", value="base-search-card__title")
    job_titles += [element.text for element in job_title_elements]

    job_company_elements = driver.find_elements(by="class name", value="base-search-card__subtitle")
    job_companies += [element.text for element in job_company_elements]

    job_location_elements = driver.find_elements(by="class name", value="job-search-card__location")
    job_locations += [element.text for element in job_location_elements]

    job_link_elements = driver.find_elements(by="class name", value="base-card__full-link")
    job_links += [element.get_attribute("href") for element in job_link_elements]

    print(f'Total Jobs Extracted: {len(job_titles)}')
    print(f'Total titles: {len(job_title_elements)}')
    print(f'Total companies: {len(job_company_elements)}')
    print(f'Total locations: {len(job_location_elements)}')
    print(f'Total links: {len(job_link_elements)}')
    
    
    # If we've collected enough jobs, break the loop
    if len(job_titles) >= total_jobs:
        print("Reached the desired number of jobs. Stopping...")
        break

    # append what we scraped till now to the `jobs.csv` file and save this file to disk
    jobsss = {
        'Job Title': job_titles,
        'Company': job_companies,
        'Location': job_locations,
        'Link': job_links,
    }
    df = pd.DataFrame.from_dict(jobsss, orient='index')
    df = df.transpose()
    
    df.to_csv('jobs.csv', index=False)
    
    
    try:
        # Use WebDriverWait with ExpectedConditions to wait for the button to be clickable
        button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "infinite-scroller__show-more-button"))
        )
        button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "infinite-scroller__show-more-button"))
        )
        button.click()
        print("Clicking 'Show More' button...")
    except (NoSuchElementException, ElementNotInteractableException):
        # No more "Show More" button or button not interactable, break the loop
        print("No more 'Show More' button. Stopping...")
        break


# # Extract job descriptions
# for job_link in job_links:
#     driver.get(job_link)
    
#     # Handle authentication wall
#     for _ in range(3):  # Try a few times
#         try:
#             show_more_button = WebDriverWait(driver, 10).until(
#                 EC.element_to_be_clickable((By.CLASS_NAME, "show-more-less-html__button"))
#             )
#             show_more_button.click()
#             print("Clicking 'Show more' button for job description...")
            
#             # Wait for the job description to load
#             time.sleep(2)
            
#             description_element = driver.find_element(By.CLASS_NAME, "show-more-less-html__markup")
#             job_descriptions.append(description_element.text)
#             print("Job description extracted.")
#             break  # Exit the loop if successful
#         except TimeoutException:
#             job_descriptions.append("N/A")
#             print("TimeoutException: No 'Show more' button found for job description within the specified timeout.")
#             # If authentication wall detected, refresh the page
#             if "authwall" in driver.current_url:
#                 print("Authentication wall detected. Pausing for 5 seconds...")
#                 time.sleep(5)
#                 print("Authentication wall detected. Searching the job again...")
#                 driver.get(job_link)
#         except (NoSuchElementException, ElementNotInteractableException):
#             job_descriptions.append("N/A")
#             print("No 'Show more' button found for job description.")
#             break  # Exit the loop if the button is not found.

# # Trim lists to the desired length
# job_titles = job_titles[:total_jobs]
# job_companies = job_companies[:total_jobs]
# job_locations = job_locations[:total_jobs]
# job_links = job_links[:total_jobs]
# job_descriptions = job_descriptions[:total_jobs]

# # Store in DataFrame
# df = pd.DataFrame({
#     'Job Title': job_titles,
#     'Company': job_companies,
#     'Location': job_locations,
#     'Link': job_links,
# })

# # Save to CSV
# df.to_csv('jobs_with_description.csv', index=False)

# Close the WebDriver
driver.quit()

print("Job scraping completed. Check 'jobs_with_description.csv' for the results.")


The msedgedriver version (120.0.2210.133) detected in PATH at d:\Project\Python_Scraping\msedgedriver.exe might not be compatible with the detected MicrosoftEdge version (121.0.2277.49); currently, msedgedriver 121.0.2277.49 is recommended for MicrosoftEdge 121.*, so it is advised to delete the driver in PATH and retry


Scraping jobs...
Total Jobs Extracted: 175
Clicking 'Show More' button...
Total Jobs Extracted: 375
Reached the desired number of jobs. Stopping...


ValueError: All arrays must be of the same length

In [4]:
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

# Ask for user input
location = input("Enter the location of the file: ")
jobtype = input("Enter the job type: ")
total_jobs = int(input("Enter the number of jobs to scrape: "))
output_file = input("Enter the output file name(only name with .csv): ")

url = f"https://www.linkedin.com/jobs/{jobtype}-jobs/?location={location}"


# Set up the WebDriver with the start-maximized option
options = webdriver.EdgeOptions()
options.add_argument("start-maximized")
driver = webdriver.Edge(options=options)



driver.get(url)


job_titles = []
job_companies = []
job_locations = []
job_links = []
# job_descriptions = []

print("Scraping jobs...")


# Infinite scroll
while len(job_titles) < total_jobs:
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        print("Scrolling to the bottom of the page...")
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait for page to load
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        if new_height == last_height:
            break
        
        last_height = new_height

    # Parse page source with BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Extract data
    job_title_elements = driver.find_elements(by="class name", value="base-search-card__title")
    job_titles += [element.text for element in job_title_elements]

    job_company_elements = driver.find_elements(by="class name", value="base-search-card__subtitle")
    job_companies += [element.text for element in job_company_elements]

    job_location_elements = driver.find_elements(by="class name", value="job-search-card__location")
    job_locations += [element.text for element in job_location_elements]

    job_link_elements = driver.find_elements(by="class name", value="base-card__full-link")
    job_links += [element.get_attribute("href") for element in job_link_elements]

    print(f'Total Jobs Extracted: {len(job_titles)}')
    print(f'Total titles: {len(job_title_elements)}')
    print(f'Total companies: {len(job_company_elements)}')
    print(f'Total locations: {len(job_location_elements)}')
    print(f'Total links: {len(job_link_elements)}')

    # append what we scraped till now to the `jobs.csv` file and save this file to disk
    print("Saving to CSV...")
    jobsss = {
        'Job Title': job_titles,
        'Company': job_companies,
        'Location': job_locations,
        'Link': job_links,
    }
    df = pd.DataFrame.from_dict(jobsss, orient='index')
    df = df.transpose()
    
    # Use mode='a' to append to the existing file
    csv_file = f'{output_file}.csv'
    df.to_csv(csv_file, mode='a', header=False, index=False)
    print(f'Saved to CSV. Total jobs: {len(df)}')

    try:
        # Use WebDriverWait with ExpectedConditions to wait for the button to be clickable
        button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "infinite-scroller__show-more-button"))
        )
        button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "infinite-scroller__show-more-button"))
        )
        button.click()
        print("Clicking 'Show More' button...")
    except (NoSuchElementException, ElementNotInteractableException):
        # No more "Show More" button or button not interactable, break the loop
        print("No more 'Show More' button. Stopping...")
        break

# Close the WebDriver
driver.quit()

print("Job scraping completed. Check 'jobs.csv' for the results.")



The msedgedriver version (120.0.2210.133) detected in PATH at d:\Project\Python_Scraping\msedgedriver.exe might not be compatible with the detected MicrosoftEdge version (121.0.2277.49); currently, msedgedriver 121.0.2277.49 is recommended for MicrosoftEdge 121.*, so it is advised to delete the driver in PATH and retry


Scraping jobs...
Scrolling to the bottom of the page...
Total Jobs Extracted: 15
Total titles: 15
Total companies: 15
Total locations: 15
Total links: 15
Saving to CSV...
Saved to CSV. Total jobs: 15


TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF723790332+58194]
	Microsoft::Applications::Events::ILogConfiguration::operator* [0x00007FF72371EC92+216482]
	(No symbol) [0x00007FF723539E1B]
	(No symbol) [0x00007FF723589C6F]
	(No symbol) [0x00007FF723589DA5]
	(No symbol) [0x00007FF7235CBB27]
	(No symbol) [0x00007FF7235AC11F]
	(No symbol) [0x00007FF72357D3E1]
	(No symbol) [0x00007FF7235C8A7E]
	(No symbol) [0x00007FF7235ABD43]
	(No symbol) [0x00007FF72357C162]
	(No symbol) [0x00007FF72357B5BD]
	(No symbol) [0x00007FF72357C994]
	Microsoft::Applications::Events::EventProperty::to_string [0x00007FF723946CDA+1051354]
	(No symbol) [0x00007FF7235FF596]
	Microsoft::Applications::Events::EventProperty::clear [0x00007FF723696215+34245]
	Microsoft::Applications::Events::EventProperty::clear [0x00007FF72368E6BD+2669]
	Microsoft::Applications::Events::EventProperty::to_string [0x00007FF7239459D3+1046483]
	Microsoft::Applications::Events::ILogConfiguration::operator* [0x00007FF723